In [37]:
from datetime import datetime
import json
import sys
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.utils import encode_dss_signature
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature

# Helper function for converting strings into byte arrays needed by cryptographic functions
def string_to_bytes(s):
    return s.encode('utf-8')

# This function will ensure that we represent the JSON dictionary as exactly the
# same string every time, otherwise we'd get different hashes while signing
def canonicalize_json(j):
    return json.dumps(j, sort_keys=True)

def verify(ca_identity, signed_message_filename): 

    print("Trying to verify " + signed_message_filename)

    # Load the signed message data
    with open(signed_message_filename, 'r') as fh:
        signed_message = json.load(fh)

    # Read out the identity of the signer and load their certificate
    signer_identity = signed_message['signer identity']
    with open(signer_identity + '.cert', 'r') as fh:
        signer_cert = json.load(fh)
    # Format the certificate body for signing as a byte array in a canonical order
    cert_body_to_be_signed = string_to_bytes(canonicalize_json(signer_cert["body"]))

    # Read out the identity of the issuer and load their public key
    issuer_identity = signer_cert['body']['issuer identity']
    signer_pk = serialization.load_pem_public_key(string_to_bytes(signer_cert['body']['public key']))
    with open(ca_identity + '.pk', 'r') as fh:
        ca_public_key = serialization.load_pem_public_key(string_to_bytes(fh.read()))

    # YOUR SOLUTION STARTS HERE
    msg_body = signed_message["message"]
    msg_sig_raw = signed_message["signature"]
    msg_sig = encode_dss_signature(msg_sig_raw["r"], msg_sig_raw["s"])
    ecdsa = ec.ECDSA(hashes.SHA256())
    try:
        # Check that the message is authentic
        signer_pk.verify(msg_sig, string_to_bytes(msg_body), ecdsa)
    except InvalidSignature:
        print(f"The message is not authentic against {signer_identity}'s public key")
        return False

    validity_start = datetime.fromisoformat(signer_cert["body"]["validity start"])
    validity_end = datetime.fromisoformat(signer_cert["body"]["validity end"])
    if datetime.now() < validity_start or datetime.now() > validity_end:
        print("Certificate has expired")
        return False
    if issuer_identity != "dstebila":
        print("Certificate is not signed by dstebila")
        return False
    cert_sig = encode_dss_signature(
        signer_cert["signature"]["r"],
        signer_cert["signature"]["s"],
    )
    try:
        ca_public_key.verify(cert_sig, cert_body_to_be_signed, ecdsa)
    except InvalidSignature:
        print(f"Certificate is not authentic")
        return False

    print("Message can be trusted")

verify("dstebila", "message1.signed.txt")
verify("dstebila", "message2.signed.txt")
verify("dstebila", "message3.signed.txt")
verify("dstebila", "message4.signed.txt")
verify("dstebila", "message5.signed.txt")


Trying to verify message1.signed.txt
Message can be trusted
Trying to verify message2.signed.txt
The message is not authentic against jeanne's public key
Trying to verify message3.signed.txt
Certificate is not authentic
Trying to verify message4.signed.txt
Certificate has expired
Trying to verify message5.signed.txt
Certificate is not signed by dstebila


False